In [ ]:
#final diet program

import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load the dataset
try:
    df = pd.read_csv('final_indian_food_cleaned.csv')
except FileNotFoundError:
    print("Error: File not found.")
    df = None

if df is not None:
    # Handle missing values
    df.fillna(df.median(numeric_only=True), inplace=True)
    df.dropna(inplace=True)

    # Remove duplicates
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Convert dietary preference to lowercase and strip spaces
    df['VegNovVeg'] = df['VegNovVeg'].str.lower().str.strip()

    # Define high-sugar and deep-fried food items
    high_sugar_foods = ['jalebi', 'gulab jamun', 'kheer', 'rabri', 'shrikhand', 'basundi', 'laddu', 'halwa', 'imarti', 'ghevar', 'phirni', 'sweet pongal', 'rasgulla', 'payasam', 'mysore pak', 'chikki', 'shahi tukda', 'gajar halwa']
    deep_fried_foods = ['samosa', 'kachori', 'pani puri', 'sev puri', 'vada pav', 'bhature', 'pakora', 'baida roti', 'dabeli', 'achappam', 'kodubale', 'chole bhature']

    # Define Curry, Dal, and Sambar categories
    curry_list = ['butter chicken', 'methi malai murg', 'paneer tikka', 'shahi paneer',
                  'mushroom masala', 'chicken korma', 'chettinad curry', 'keema',
                  'stuffed capsicum', 'prawn masala', 'mutton rogan josh']

    dal_sambar_list = ['dal tadka', 'dal makhani', 'masoor dal', 'pithla bhakri',
                       'chana dal', 'dal baati churma', 'sambar']

    # Function to check and add Roti or Rice
    def check_curry_or_dal(meal, meal_type):
        has_curry, has_dal_sambar = False, False
        updated_meal = meal[:]

        for item in meal:
            if item.lower() in curry_list or item.lower() == "chicken korma":
                has_curry = True
            if item.lower() in dal_sambar_list:
                has_dal_sambar = True

        if has_curry and "Roti" not in updated_meal:
            updated_meal.append("Roti")
        if has_dal_sambar and meal_type != "Breakfast" and "Plain Rice" not in updated_meal:
            updated_meal.append("Plain Rice")

        return list(set(updated_meal))  # Remove duplicates

    # Remove non-veg items if vegetarian diet is chosen
    def filter_veg(df, diet_preference):
        if diet_preference == "veg":
            df = df[df['VegNovVeg'] == 'veg']
        return df

    # Function to adjust meals based on HbA1c levels
    def filter_hba1c_friendly(df, hba1c):
        if hba1c >= 6.5:  # Diabetes
            df = df[~df['Food_items'].str.lower().isin(high_sugar_foods)]
            df = df[~df['Food_items'].str.lower().isin(deep_fried_foods)]
        elif 5.7 <= hba1c < 6.5:  # Prediabetes
            df = df[~df['Food_items'].str.lower().isin(high_sugar_foods)]
        return df

    # Get user inputs
    age = int(input("Enter your age: "))
    diet_preference = input("Choose your diet preference (veg/nonveg): ").lower()
    weight = float(input("Enter your weight (kg): "))
    height = float(input("Enter your height (cm): "))
    goal = input("Choose your goal ['weight loss', 'weight gain', 'healthy diet']: ").lower()
    has_diabetes = input("Do you have diabetes? (yes/no): ").lower()

    hba1c = None
    if has_diabetes == 'yes':
        hba1c = float(input("Enter your HbA1c level (%): "))

    # Apply strict filtering for vegetarian/non-vegetarian
    filtered_df = filter_veg(df, diet_preference)

    # Apply HbA1c-based filtering
    if has_diabetes == 'yes' and hba1c:
        filtered_df = filter_hba1c_friendly(filtered_df, hba1c)

    # Generate structured meal plan
    def generate_diet_plan():
        day_names = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        diet_plan = {}

        for day in day_names:
            diet_plan[day] = {
                'Breakfast': random.sample(filtered_df[filtered_df['Breakfast'] == 1]['Food_items'].tolist(), 3),
                'Lunch': check_curry_or_dal(random.sample(filtered_df[filtered_df['Lunch'] == 1]['Food_items'].tolist(), 3), "Lunch"),
                'Dinner': check_curry_or_dal(random.sample(filtered_df[filtered_df['Dinner'] == 1]['Food_items'].tolist(), 3), "Dinner")
            }
        return diet_plan

    # Display the meal plan
    diet_plan = generate_diet_plan()
    print("\nDiet Plan for 7 Days:\n")
    for day, meals in diet_plan.items():
        print(f"{day}:")
        for meal_type, food_items in meals.items():
            print(f"  {meal_type}: {', '.join(food_items)}")
        print()
